# Analyze PE

This notebook helps with investigations of EF files 

## Usage

You should have the suspicious file in the readonly directory mounted on the Desktop. The suspicious file should be in a password protected 7-Zip file with the password *infected*.

The suspicious file is unpacked to a created directory on the *Desktop* named *Investigations/files*. Then **capa** is run in normal mode to create a report on the the file. After that **capa** is run again to output the the results in `JSON` format and with the results in a file named with the same name as the suspicious file with an added extension of *.capa.json*. Last the file is imported to a projected named *Investigations* in **Ghidra**. The project directory is located under *C:/Users/WDAGUtilityAccount/Ghidra*.

For information on how to import the result in the **capa** `JSON` file into Ghidra read more in the wiki..

In [ ]:
# Imports
from common import execute7z, READONLY
import magic
import subprocess
from pathlib import Path

In [ ]:
# Create directory for extracted files
if not Path("C:/Users/WDAGUtilityAccount/Desktop/Investigations").is_dir():
    Path("C:/Users/WDAGUtilityAccount/Desktop/Investigations").mkdir()
    Path("C:/Users/WDAGUtilityAccount/Desktop/Investigations/files").mkdir()
    Path("C:/Users/WDAGUtilityAccount/Desktop/Investigations/json").mkdir()

In [ ]:
# Extract files
for suspicious_zip in Path(READONLY).glob("*.7z"):
    process = execute7z(suspicious_zip, "infected", "C:/Users/WDAGUtilityAccount/Desktop/Investigations/files")
    if process == 0:
        print("Extracted files from:", suspicious_zip)
    else:
        print("Failed to extract file")

In [ ]:
# Check file type
for suspicious_file in Path("C:/Users/WDAGUtilityAccount/Desktop/Investigations/files/").glob("*.*"):
    print(suspicious_file, end=": ")
    print(magic.from_file(str(suspicious_file)))

In [ ]:
# Run capa on files
for suspicious_file in Path("C:/Users/WDAGUtilityAccount/Desktop/Investigations/files/").glob("*.*"):
    capa_result = subprocess.run(["capa.exe", suspicious_file], shell=True, stdout=subprocess.PIPE)
    print(capa_result.stdout.decode('utf-8'))

In [ ]:
# Run capa on files with json output
for suspicious_file in Path("C:/Users/WDAGUtilityAccount/Desktop/Investigations/files/").glob("*.*"):
    capa_result = subprocess.run(["capa-ghidra.exe", "-j", suspicious_file], shell=True, stdout=subprocess.PIPE)
    with open(str(suspicious_file).replace('\\files\\','\\json\\') + ".capa.json", "w", encoding="utf-8") as outfile:
        outfile.writelines(capa_result.stdout.decode('utf-8'))

In [ ]:
# Ensure there is a directory for Ghidra projects
if not Path("C:/Users/WDAGUtilityAccount/Ghidra").is_dir():
    Path("C:/Users/WDAGUtilityAccount/Ghidra").mkdir()

In [ ]:
# Import suspicious file to Ghidra in headless mode and then start Ghidra with the investigation project
for suspicious_file in Path("C:/Users/WDAGUtilityAccount/Desktop/Investigations/files/").glob("*.*"):
    result_import = subprocess.run(
        [
            "C:/Tools/ghidra/support/analyzeHeadless",
            "C:/Users/WDAGUtilityAccount/Ghidra",
            "investigation",
            "-import", suspicious_file
        ],
        shell=True, check=True
    )
    result_run = subprocess.run(
        [
            "C:\Tools\ghidra\ghidraRun.bat",
            "C:/Users/WDAGUtilityAccount/Ghidra/investigation.gpr"
        ],
        shell=True, check=True
    )